<a href="https://colab.research.google.com/github/cjimenezrosas/Datasets/blob/main/Soluci%C3%B3n_SCP_usando_SCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Factibilidad

In [ ]:
import numpy as np
def check_sol(SCP, sol):
    flag = True
    set = SCP["S"]
    aux = np.dot(set,sol)           # Multiplico la matriz de cobertura por la solucion

    if 0 in aux:                    # Si alguna zona no esta cubierta, retorno False
        flag = False

    return flag, aux

Leer problema

In [ ]:
def read_instance(Instancia):

    file = open(Instancia, "r")

    ### Genero una lista con todos los valores del archivo. Se almacenan en u.
    lines = file.readlines()
    lines = [line.replace('\n', ' ').strip().split(" ") for line in lines]
    u = []
    for item in lines:
        u += item
    u = list(map(int, u))

    # Leer Dimensión
    # Con la función pop() retorno el primer elemento de la lista u, y se elimina de la lista.
    m = u.pop(0)
    n = u.pop(0)
    # print("Número de restricciones: "+str(m))
    # print("Número de columnas: "+str(n))
    # print("Dimensiones de la matriz de cobertura: ["+str(m)+" , "+str(n)+"]")

    
    # Leer Costo
    # Obtengo los costos de cada subset (columna)

    C = [int(i) for i in u[:n]]
    Cnp = np.array(C)
    # print("Costos para cada columna: "+str(Cnp.tolist()))
    # print("Cantidad de costos almacenados: "+str(Cnp.__len__()))

    # Elimino la lista de costos de u
    u = u[n:]

    # Verifico la correcta lectura de las variables
    # print(n, m, len(C), u[0])

    # Leer Restricciones
    SS = []
    c_s = []
    set = np.zeros((m,n))
    for j in range(m):
        # Obtengo el tamaño del subset
        aux = u.pop(0)
        # print("Cantidad de columnas que cubre la restriccion "+str(j)+": "+str(aux))

        # Capturo el subset y le resto 1 a cada valor, para usarlo como índice
        s = u[:aux]
        s = list(map(lambda elemento:elemento-1, s))
        # print("Columnas que cumbren la restriccion "+str(j)+": "+str(s))

        # Actualizo cobertura, coloco los 1 en las posiciones correspondientes
        # print("Antes de colocar unos: "+str(set[j,s].tolist()))
        set[j,s] = 1
        # print("Despues de colocar unos: "+str(set[j,s].tolist()))

        # Arego a la lista de subsets
        SS.append(s)

        # Elimino el subset de u
        u = u[aux:]

    # print(set)

    # validacion de colocacion de 1 
    # print("Validacion de matriz de cobertura: "+str(set[0][90]))

    SCP        = {}
    SCP["n"]   = n          # Numero de subsets / coberturas / columnas
    SCP["m"]   = m          # Numero de restricciones / zonas
    SCP["C_s"] = Cnp        # Costo de seleccionar cada subset / cobertura
    SCP["S"]   = set        # Subsets de la instancia - matriz de cobertura

    return SCP

Fitness

In [ ]:
import numpy as np
from cmath import inf

def fitness_function(SCP, sol):
    cost = SCP["C_s"]
    feasible = check_sol(SCP, sol)[0]
    if not feasible:
        return inf
    else:
        fitness_cal = np.dot(sol, cost)
        return fitness_cal

Binarización

In [ ]:
import numpy as np
from scipy import special as scyesp

class DiscretizationScheme:
    
    def __init__(self, matrixCont, matrixBin, SolutionRanking, transferFunction, binarizationOperator):

        self.transferFunction = transferFunction
        self.binarizationOperator = binarizationOperator

        self.matrixCont = matrixCont
        self.matrixBin = matrixBin
        self.SolutionRanking = SolutionRanking
        self.bestRow = np.argmin(SolutionRanking) 

        #output
        self.matrixProbT = np.zeros(self.matrixCont.shape)
        self.matrixBinOut = np.zeros(self.matrixBin.shape)

        # debug
        self.matrixProbTAux = np.zeros(self.matrixCont.shape)
        self.matrixBinOutAux = np.zeros(self.matrixBin.shape)

        #Constantes O1
        self.o1a= 0
        self.o1b= 1
        self.o1c= 1
        self.o1d= 0
        
        #Constante de Q-Shaped
        self.xMax = np.max(0.5*self.matrixCont)

        self.uAlpha = 0.5 #*** Por definir como ingresar
        self.uBeta = 1.5 #*** Por definir como ingresar


    #Funciones de Transferencia
    def T_V1(self):
        self.matrixProbT = np.abs(scyesp.erf(np.divide(np.sqrt(np.pi),2)*self.matrixCont))

    def T_V2(self):
        self.matrixProbT = np.abs(np.tanh(self.matrixCont))

    def T_V3(self):
        self.matrixProbT = np.abs(np.divide(self.matrixCont, np.sqrt(1+np.power(self.matrixCont,2))))

    def T_V4(self):
        self.matrixProbT = np.abs( np.divide(2,np.pi)*np.arctan( np.divide(np.pi,2)*self.matrixCont ))

    def T_S1(self):
        self.matrixProbT = np.divide(1, ( 1 + np.exp(-2*self.matrixCont) ) )

    def T_S2(self):
        self.matrixProbT = np.divide(1, ( 1 + np.exp(-1*self.matrixCont) ) )

    def T_S3(self):
        self.matrixProbT = np.divide(1, ( 1 + np.exp(np.divide(-1*self.matrixCont,2) ) ))

    def T_S4(self):
        self.matrixProbT = np.divide(1, ( 1 + np.exp(np.divide(-1*self.matrixCont,3) ) ))

    def T_O1(self):
        self.matrixProbT = np.sin(np.multiply(np.multiply(np.multiply(np.multiply(2,np.pi),(self.matrixCont - self.o1a)),self.o1b),np.cos(np.multiply(np.multiply(np.multiply(2,np.pi),(self.matrixCont - self.o1a)),self.o1c))))+ self.o1d

    def T_O2(self): #*** solamente entregaría 1 o 0
        self.matrixProbT =np.trunc(np.abs(np.mod(self.matrixCont,2)))

    def T_O3(self):
        self.matrixProbT = np.divide((self.matrixCont + np.min(self.matrixCont)),(np.abs(np.min(self.matrixCont))+np.max(self.matrixCont)))

    def T_O4(self): #*** está fuera del dominio [0,1]
        self.matrixProbT = self.matrixCont

    def T_Q1(self): 
        self.matrixProbT[self.matrixCont < self.xMax] = np.abs(np.divide(self.matrixCont[self.matrixCont < self.xMax],np.max(0.5*self.matrixCont[self.matrixCont < self.xMax])))
        self.matrixProbT[self.matrixCont >= self.xMax] = 1

    def T_Q2(self):
        self.matrixProbT[self.matrixCont < self.xMax] = np.power(np.divide(self.matrixCont[self.matrixCont < self.xMax],np.max(0.5*self.matrixCont[self.matrixCont < self.xMax])),2)
        self.matrixProbT[self.matrixCont >= self.xMax] = 1

    def T_Q3(self): 
        self.matrixProbT[self.matrixCont < self.xMax] = np.power(np.divide(self.matrixCont[self.matrixCont < self.xMax],np.max(0.5*self.matrixCont[self.matrixCont < self.xMax])),3)
        self.matrixProbT[self.matrixCont >= self.xMax] = 1

    def T_Q4(self): 
        self.matrixProbT[self.matrixCont < self.xMax] = np.power(np.divide(self.matrixCont[self.matrixCont < self.xMax],np.max(0.5*self.matrixCont[self.matrixCont < self.xMax])),4)
        self.matrixProbT[self.matrixCont >= self.xMax] = 1

    # def T_U(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
    #     self.matrixProbT = self.uAlpha*(np.power(np.abs(self.matrixCont),self.uBeta))

    def T_U1(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 0.5*(np.power(np.abs(self.matrixCont),1.5))

    def T_U2(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 0.5*(np.power(np.abs(self.matrixCont),2.75))

    def T_U3(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 0.5*(np.power(np.abs(self.matrixCont),4))

    def T_U4(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 1.25*(np.power(np.abs(self.matrixCont),1.5))

    def T_U5(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 1.25*(np.power(np.abs(self.matrixCont),2.75))

    def T_U6(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 1.25*(np.power(np.abs(self.matrixCont),4))

    def T_U7(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 2*(np.power(np.abs(self.matrixCont),1.5))

    def T_U8(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 2*(np.power(np.abs(self.matrixCont),2.75))

    def T_U9(self): #Hay que ver como definir como ingresar self.uAlpha y self.uBeta
        self.matrixProbT = 2*(np.power(np.abs(self.matrixCont),4))

    def T_Z1(self): 
        self.matrixProbT = np.power((1-np.power(2,self.matrixCont)),0.5)

    def T_Z2(self): 
        self.matrixProbT = np.power((1-np.power(5,self.matrixCont)),0.5)

    def T_Z3(self): 
        self.matrixProbT = np.power((1-np.power(8,self.matrixCont)),0.5)

    def T_Z4(self): 
        self.matrixProbT = np.power((1-np.power(20,self.matrixCont)),0.5)

    def T_X1(self): #funciones de transferencias S1 invertidad
        self.matrixProbT = np.divide(1, ( 1 + np.exp(2*self.matrixCont) ) )

    def T_X2(self): #funciones de transferencias S2 invertidad
        self.matrixProbT = np.divide(1, ( 1 + np.exp(1*self.matrixCont) ) )

    def T_X3(self): #funciones de transferencias S3 invertidad
        self.matrixProbT = np.divide(1, ( 1 + np.exp(np.divide(1*self.matrixCont,2) ) ))

    def T_X4(self): #funciones de transferencias S4 invertidad
        self.matrixProbT = np.divide(1, ( 1 + np.exp(np.divide(1*self.matrixCont,3) ) ))

    #Binarization
    def B_Standard(self):
        matrixRand = np.random.uniform(low=0.0,high=1.0,size=self.matrixCont.shape)
        self.matrixBinOut = np.greater(self.matrixProbT,matrixRand).astype(int)

    def B_Complement(self):
        matrixRand = np.random.uniform(low=0.0,high=1.0,size=self.matrixCont.shape)
        matrixComplement = np.abs(1-self.matrixBin)
        self.matrixBinOut = np.multiply(np.greater_equal(self.matrixProbT,matrixRand).astype(int),matrixComplement)

    def B_Elitist(self):
        matrixRand = np.random.uniform(low=0.0,high=1.0,size=self.matrixCont.shape)
        # greater, porque es estricto en la ecuacion.
        conditionMatrix = np.greater(self.matrixProbT,matrixRand)
        #todo: validar que el index exista
        bestIndividual = self.matrixBin[self.bestRow]
        # si ProbT > Rand() , then bestIndividualBin, else 0
        self.matrixBinOut = np.where(conditionMatrix==True,bestIndividual,0)

    def B_Static(self):
        alfa = 1/3
        self.matrixBinOut[self.matrixProbT<=alfa] = 0
        self.matrixBinOut[(self.matrixProbT > alfa) & (self.matrixProbT <= 0.5*(1+alfa))] = self.matrixBin[(self.matrixProbT > alfa) & (self.matrixProbT <= 0.5*(1+alfa))]
        self.matrixBinOut[self.matrixProbT>=0.5*(1+alfa)] = 1

    def B_ElitistRoulette(self):
        matrixRand = np.random.uniform(low=0.0,high=1.0,size=self.matrixCont.shape)
        #greater, porque es estricto en la ecuacion.
        conditionMatrix = np.greater(self.matrixProbT,matrixRand)
        #todo: validar que el index exista
        alfa = 0.2
        # condicion sum()==0, para el caso en que entregamos lista de rank con [0 0 0 0 0 0 0 0 ... 0 0 0 0] -> cuando generamos poblacion inicial
        if (self.SolutionRanking.shape[0]*alfa < 1) or (self.SolutionRanking.sum()==0):
            bestIndividual = self.matrixBin[0]
        else:
            BestSolutionRaking = int(self.SolutionRanking.shape[0] * alfa)
            random = np.random.randint(low = 0, high = BestSolutionRaking)
            bestIndividual = self.matrixBin[random]
        self.matrixBinOut = np.where(conditionMatrix==True,bestIndividual,0)

    def binariza(self):
        if self.transferFunction == 'V1':
            self.T_V1()

        if self.transferFunction == 'V2':
            self.T_V2()

        if self.transferFunction == 'V3':
            self.T_V3()

        if self.transferFunction == 'V4':
            self.T_V4()

        if self.transferFunction == 'S1':
            self.T_S1()

        if self.transferFunction == 'S2':
            self.T_S2()

        if self.transferFunction == 'S3':
            self.T_S3()

        if self.transferFunction == 'S4':
            self.T_S4()

        if self.transferFunction == 'O1':
            self.T_O1()

        if self.transferFunction == 'Z1':
            self.T_Z1()

        if self.transferFunction == 'Z2':
            self.T_Z2()

        if self.transferFunction == 'Z3':
            self.T_Z3()

        if self.transferFunction == 'Z4':
            self.T_Z4()

        if self.transferFunction == 'X1':
            self.T_X1()

        if self.transferFunction == 'X2':
            self.T_X2()

        if self.transferFunction == 'X3':
            self.T_X3()

        if self.transferFunction == 'X4':
            self.T_X4()

        if self.binarizationOperator == 'Standard':
            self.B_Standard()

        if self.binarizationOperator == 'Complement':
            self.B_Complement()

        if self.binarizationOperator == 'Elitist':
            self.B_Elitist()

        if self.binarizationOperator == 'Static':
            self.B_Static()

        if self.binarizationOperator == 'ElitistRoulette':
            self.B_ElitistRoulette()

        return self.matrixBinOut

    def appliedTransferFunction(self):
        if self.transferFunction == 'V1':
            self.T_V1()

        if self.transferFunction == 'V2':
            self.T_V2()

        if self.transferFunction == 'V3':
            self.T_V3()

        if self.transferFunction == 'V4':
            self.T_V4()

        if self.transferFunction == 'S1':
            self.T_S1()

        if self.transferFunction == 'S2':
            self.T_S2()

        if self.transferFunction == 'S3':
            self.T_S3()

        if self.transferFunction == 'S4':
            self.T_S4()

        if self.transferFunction == 'O1':
            self.T_O1()

        if self.transferFunction == 'O2':
            self.T_O2()

        if self.transferFunction == 'O3':
            self.T_O3()

        if self.transferFunction == 'O4':
            self.T_O4()

        if self.transferFunction == 'Q1':
            self.T_Q1()

        if self.transferFunction == 'Q2':
            self.T_Q2()

        if self.transferFunction == 'Q3':
            self.T_Q3()

        if self.transferFunction == 'Q4':
            self.T_Q4()

        if self.transferFunction == 'Z1':
            self.T_Z1()

        if self.transferFunction == 'Z2':
            self.T_Z2()

        if self.transferFunction == 'Z3':
            self.T_Z3()

        if self.transferFunction == 'Z4':
            self.T_Z4()

        if self.transferFunction == 'X1':
            self.T_X1()

        if self.transferFunction == 'X2':
            self.T_X2()

        if self.transferFunction == 'X3':
            self.T_X3()

        if self.transferFunction == 'X4':
            self.T_X4()

        return self.matrixProbT

Reparación

In [ ]:
def reparaComplejo(SCP, solution):
    sol =  np.reshape(solution, (SCP["n"],))
    set = SCP["S"]
    
    feasible, aux = check_sol(SCP,sol)

    while not feasible:
        r_no_cubiertas = np.zeros((SCP["m"],))
        r_no_cubiertas[np.argwhere(aux == 0)] = 1           # Vector indica las restricciones no cubiertas
        cnc = np.dot(r_no_cubiertas, set)                   # Cantidad de restricciones no cubiertas que cubre cada columna (de tamaño n)
        trade_off = np.divide(SCP["C_s"],cnc)               # Trade off entre zonas no cubiertas y costo de seleccionar cada columna
        idx = np.argmin(trade_off)                          # Selecciono la columna con el trade off mas bajo
        sol[idx] = 1                                        # Asigno 1 a esa columna
        feasible, aux = check_sol(SCP,sol)               # Verifico si la solucion actualizada es factible

    return sol

Algortimo Sine Cosine

In [ ]:
from ctypes import util
import random
import math
import numpy as np

#---------------------------------------------------------------------------SINE COSINE ALGORITHM---------------------------------------------------------------------------------#
# esta funcion recibe como parametros:
# total de iteraciones: maxIter
# iteracion actual: t
# dimensiones del problema: dimension
# la poblacion completa CONTINUA: poblacion
# mejor individuo de la poblacion: bestSolutionCon
def iterarSCA(maxIter, t, dimension, poblacion, bestSolutionCon):
    # nuestro valor de a es constante y es extraido desde paper
    a = 2
    # calculamos nuestro r1
    r1 = a - (t * (a / maxIter))
    # for de individuos
    for i in range(poblacion.__len__()):
        # for de dimensiones
        for j in range(dimension):
            # calculo un numero aleatoreo entre [0,1]
            rand = random.uniform(0.0, 1.0)
            # calculo r2
            r2 =  (2 * math.pi) * rand 
            # calculo r3
            r3 = 2 * rand
            # calculo r4
            r4 = random.uniform(0.0, 1.0)
            if r4 < 0.5:
                # perturbo la poblacion utilizando como base la funcion seno
                poblacion[i][j] = poblacion[i][j] + ( ( ( r1 * math.sin(r2)) * abs( ( r3 * bestSolutionCon[j] ) - poblacion[i][j] ) ) )
            else:
                # perturbo la poblacion utilizando como base la funcion coseno
                poblacion[i][j] = poblacion[i][j] + ( ( ( r1 * math.cos(r2)) * abs( ( r3 * bestSolutionCon[j] ) - poblacion[i][j] ) ) )
    # retorno la poblacion modificada 
    return np.array(poblacion)




**Solución**

In [ ]:
from pickletools import int4
import numpy as np
import time
import math
import sys
from numpy import linalg as LA


# tiempo inicial para la ejecucion completa
tiempoInicial = time.time()

# ruta instancias
dirSCP = '/content/drive/MyDrive/SCP/scpnre3.txt'
dirResultado = '/content/drive/MyDrive/Resultados'

print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("Resolviendo Problema: "+dirSCP.split("/")[5].split(".")[0])


mh = "Sine Cosine Algorithm"
problema = dirSCP.split("/")[5].split(".")[0]

resultado = open(dirResultado+mh+"_"+problema+".txt", "w")


# lectura de la instancia
SCP = read_instance(dirSCP)

# matriz de cobertura m x n
matrizCobertura = SCP["S"]    
# vector de costos para cada columna n
vectorCostos = SCP["C_s"]

# obtengo las dimensiones de mi problema (m)
dim = len(vectorCostos)
# tamaño de la poblacion 
pob = 20
# numero de iteraciones a iterar
maxIter = 500
# inicializo un diccionario
params = {}

# BINARIZACIÓN
# Tipos de función de transferencia: S1, S2, S3, S4, V1, V2, V3, V4, Z1, Z2, Z3, Z4
# Tipos discretización: Standard, Complement, Elitist, Static, ElitistRoulette

DS = ['S1','ElitistRoulette'] # SE ELIGE LA FUNCIÓN DE TRANSFERENCIA Y LA DISCRETIZACIÓN

# guardo en mi diccionario todos las cosas que necesito para iterar, costos, matriz de cobertura, como voy a binarizar y como voy a reparar las soluciones infactibles
params["costos"] = vectorCostos
params["cobertura"] = matrizCobertura
params["ds"] = DS
params["repairType"] = 1

# Generar población inicial
poblacion = np.random.uniform(low=-10.0, high=10.0, size=(pob,dim))

# Genero una población inicial binaria, esto ya que nuestro problema es binario (SCP)
matrixBin = np.random.randint(low=0, high=3, size = (pob,dim))
# Genero un vector donde almacenaré los fitness de cada individuo
fitness = np.zeros(pob)

# Genero un vetor dedonde tendré mis soluciones rankeadas
solutionsRanking = np.zeros(pob)

# calculo de factibilidad de cada individuo y calculo del fitness inicial
for i in range(poblacion.__len__()):
    flag, aux = check_sol(SCP,matrixBin[i])
    if not flag: #solucion infactible
        matrixBin[i] = reparaComplejo(SCP, matrixBin[i])
        
    fitness[i] = fitness_function(SCP, matrixBin[i])  

solutionsRanking = np.argsort(fitness) # rankings de los mejores fitnes
bestRowAux = solutionsRanking[0]
# mostramos nuestro fitness iniciales
print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("fitness incial: "+str(fitness))
print("Best fitness inicial: "+str(np.min(fitness)))
print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("COMIENZA A TRABAJAR LA METAHEURISTICA "+mh)
print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
# for de iteraciones
for iter in range(0, maxIter):
    
    # obtengo mi tiempo inicial
    processTime = time.process_time()  
    timerStart = time.time()


    # DETERMINO MI MEJOR SOLUCION Y LA GUARDO 
    Best = poblacion[bestRowAux]
    BestBinary = matrixBin[bestRowAux]
    BestFitness = np.min(fitness)
    
    # perturbo la poblacion con la metaheuristica
    # en las funciones internas tenemos los otros dos for, for de individuos y for de dimensiones
    poblacion = iterarSCA(maxIter, iter, dim, poblacion.tolist(), Best.tolist())
    
    # Binarizo, calculo de factibilidad de cada individuo y calculo del fitness
    for i in range(poblacion.__len__()):
        ds = DiscretizationScheme(poblacion,matrixBin,solutionsRanking,DS[0],DS[1])

        matrixBin = ds.binariza()

        flag, aux = check_sol(SCP,matrixBin[i])
        if not flag: #solucion infactible
            matrixBin[i] = reparaComplejo(SCP, matrixBin[i])

        fitness[i] = fitness_function(SCP, matrixBin[i])

        # #Conservo el Best
        # if fitness[i] < BestFitness:
        #     BestFitness = fitness[i]
        #     BestBinary = matrixBin[i]
        #     bestRowAux = i
    solutionsRanking = np.argsort(fitness) # rankings de los mejores fitnes

    #Conservo el Best
    if fitness[bestRowAux] > BestFitness:
        fitness[bestRowAux] = BestFitness
        matrixBin[bestRowAux] = BestBinary
    BestFitnes = np.min(fitness)

    timerFinal = time.time()
    # calculo mi tiempo para la iteracion t
    timeEjecuted = timerFinal - timerStart
    print("iteracion: "+str(iter)+", best fitness: "+str(BestFitness)+", tiempo iteracion (s): "+str(timeEjecuted))
    resultado.write("iteracion: "+str(iter)+", best fitness: "+str(BestFitness)+", tiempo iteracion (s): "+str(timeEjecuted)+"\n")

print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("Best fitness: "+str(BestFitness))
resultado.write("Best fitness: "+str(BestFitness)+"\n")
print("Cantidad de columnas seleccionadas: "+str(sum(BestBinary)))
resultado.write("Cantidad de columnas seleccionadas: "+str(sum(BestBinary))+"\n")
print("Best solucion: \n"+str(BestBinary.tolist()))
resultado.write("Best solucion: \n"+str(BestBinary.tolist())+"\n")
print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
tiempoFinal = time.time()
tiempoEjecucion = tiempoFinal - tiempoInicial
print("Tiempo de ejecucion (s): "+str(tiempoEjecucion))
resultado.write("Tiempo de ejecucion (s): "+str(tiempoEjecucion))
resultado.close()
print("--------------------------------------------------------------------------------------------------------------------------------------")